In [43]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict, Tuple

In [44]:
# Load configuration from config.json
with open('../models/config.json', 'r') as f:
    config = json.load(f)

postprocess = config['postprocessing']

# Output and input paths from configuration
OUTPUT_PATH = postprocess['output_path']
INPUT_PATH_S = postprocess['input_path_s']
INPUT_PATH_D = postprocess['input_path_d']

# New Input paths
INPUT_PATH_WS = postprocess['input_path_ws']
INPUT_PATH_HD = postprocess['input_path_hd']
# Bitte erweiter den Skriopt um folgende Funktionalität: Ich will den gewichteten Zielfunktionswert für jeden Tag berechnen und ausgeben.
# In INPUT_PATH_WS liegt file: d_scenarios_20230626_to_20230626_day_obj.csv
# d_scenarios_{start_date}_to_{end_date}_day_obj.csv

#Dieser hat folgende Struktur: Scenario,ObjectiveValue
# Scenario1,-115.79991236218207
# Scenario2,-125.2738754147073
# Scenario3,-98.0269803160276
# Scenario4,-106.03445229578874
# Scenario5,-110.52131413374447
# Scenario6,-109.61576138747851
# Scenario7,-117.03803729400363
# Scenario8,-136.10148198074634
# Scenario9,-112.07935001377426
# Scenario10,-112.68776316332841

# Input Path für heat demand scenarios (.json) Aus dieser Datei bekommen wir unsere Probability für jedes Szenario. Die Datei liegt im Pfad INPUT_PATH_HD
# Die Datei heißt: reduced_heat_demand_scenarios_20230626_to_20230626_day.json
# reduced_heat_demand_scenarios_{start_date}_to_{end_date}_day.json

# Diese hat folgende Struktur:

# {
#     "Scenario1": {
#         "Probability": 0.2418623085247977,
#         "1": 47.316085158528665,
#         "2": 15.342366119088688,
#         "3": 128.90399837176076,
#         "4": 19.875702035536335,
#         "5": 70.68836375127425,
#         "6": 60.89483394220605,
#         "7": 75.11244427008523,
#         "8": 22.322366635072974,
#         "9": 76.67436452309214,
#         "10": 11.252439104039858,
#         "11": 30.013131422145626,
#         "12": 7.536730580340013,
#         "13": 5.765079693539619,
#         "14": 62.54423579388769,
#         "15": 38.800487585386264,
#         "16": 43.50519539657556,
#         "17": 79.03933793590109,
#         "18": 58.94299659421233,
#         "19": 38.12994744484017,
#         "20": 25.302769887900876,
#         "21": 15.317672731983397,
#         "22": 36.095362993612085,
#         "23": 36.631603239444345,
#         "24": 40.08809430511352
#     },
#     "Scenario2": {
#         "Probability": 0.07995197538112958,
#         "1": 47.316085158528665,
#         "2": 37.44621405731088,
#         "3": 163.24303636076635,
#         "4": 81.41188739120258,
#         "5": 70.68836375127425,
#         "6": 87.14486384867749,
#         "7": 25.93390124778211,
#         "8": 48.80090849311462,
#         "9": 26.798196893359222,
#         "10": 40.380553877503864,
#         "11": 30.013131422145626,
#         "12": 54.33958361798484,
#         "13": 30.73550312555708,
#         "14": 16.16416984813666,
#         "15": 38.800487585386264,
#         "16": 43.50519539657556,
#         "17": 50.95192312182769,
#         "18": 58.94299659421233,
#         "19": 85.17520288012449,
#         "20": 25.302769887900876,
#         "21": 33.618794091973044,
#         "22": 36.095362993612085,
#         "23": 16.461206263821154,
#         "24": 19.874319556226006
#     },
#     "Scenario3": {
#         "Probability": 0.059835692794664365,
#         "1": 25.871439595195696,
#         "2": 15.342366119088688,
#         "3": 94.5649603827552,
#         "4": 19.875702035536335,
#         "5": 96.30965212639605,
#         "6": 87.14486384867749,
#         "7": 75.11244427008523,
#         "8": 48.80090849311462,
#         "9": 26.798196893359222,
#         "10": 69.50866865096788,
#         "11": -24.79772118046104,
#         "12": 7.536730580340013,
#         "13": 30.73550312555708,
#         "14": 39.35420282101217,
#         "15": 38.800487585386264,
#         "16": 71.9483150315671,
#         "17": 50.95192312182769,
#         "18": 31.11281553695568,
#         "19": 85.17520288012449,
#         "20": 25.302769887900876,
#         "21": 15.317672731983397,
#         "22": 19.045761339026384,
#         "23": -3.7091907118020373,
#         "24": 40.08809430511352
#     },
#     "Scenario4": {
#         "Probability": 0.061774261157197925,
#         "1": 4.426794031862734,
#         "2": 59.550061995533085,
#         "3": 94.5649603827552,
#         "4": 50.64379471336946,
#         "5": 96.30965212639605,
#         "6": 34.64480403573461,
#         "7": 75.11244427008523,
#         "8": 48.80090849311462,
#         "9": 51.73628070822568,
#         "10": 69.50866865096788,
#         "11": 30.013131422145626,
#         "12": 30.938157099162424,
#         "13": 30.73550312555708,
#         "14": 39.35420282101217,
#         "15": 38.800487585386264,
#         "16": 43.50519539657556,
#         "17": 79.03933793590109,
#         "18": 58.94299659421233,
#         "19": 61.65257516248233,
#         "20": 45.78120903668678,
#         "21": 15.317672731983397,
#         "22": 19.045761339026384,
#         "23": -23.879587687425243,
#         "24": -20.553229941549013
#     },
#     "Scenario5": {
#         "Probability": 0.07925989359232213,
#         "1": 25.871439595195696,
#         "2": 59.550061995533085,
#         "3": 94.5649603827552,
#         "4": 19.875702035536335,
#         "5": 70.68836375127425,
#         "6": 87.14486384867749,
#         "7": 75.11244427008523,
#         "8": 48.80090849311462,
#         "9": 51.73628070822568,
#         "10": 69.50866865096788,
#         "11": 30.013131422145626,
#         "12": 7.536730580340013,
#         "13": 55.705926557574536,
#         "14": 39.35420282101217,
#         "15": 38.800487585386264,
#         "16": 43.50519539657556,
#         "17": 22.864508307754285,
#         "18": 58.94299659421233,
#         "19": 61.65257516248233,
#         "20": 25.302769887900876,
#         "21": -2.9834486280062436,
#         "22": 1.9961596844406841,
#         "23": 16.461206263821154,
#         "24": 19.874319556226006
#     },
#     "Scenario6": {
#         "Probability": 0.07113913002121826,
#         "1": 4.426794031862734,
#         "2": 59.550061995533085,
#         "3": 128.90399837176076,
#         "4": 19.875702035536335,
#         "5": 70.68836375127425,
#         "6": 60.89483394220605,
#         "7": 50.52317275893367,
#         "8": 48.80090849311462,
#         "9": 76.67436452309214,
#         "10": 69.50866865096788,
#         "11": 57.41855772344896,
#         "12": 7.536730580340013,
#         "13": 5.765079693539619,
#         "14": 39.35420282101217,
#         "15": 38.800487585386264,
#         "16": 15.062075761584019,
#         "17": 50.95192312182769,
#         "18": 58.94299659421233,
#         "19": 38.12994744484017,
#         "20": 66.25964818547268,
#         "21": 15.317672731983397,
#         "22": 1.9961596844406841,
#         "23": 36.631603239444345,
#         "24": 19.874319556226006
#     },
#     "Scenario7": {
#         "Probability": 0.12021680952182671,
#         "1": 25.871439595195696,
#         "2": 15.342366119088688,
#         "3": 128.90399837176076,
#         "4": 50.64379471336946,
#         "5": 70.68836375127425,
#         "6": 60.89483394220605,
#         "7": 50.52317275893367,
#         "8": 22.322366635072974,
#         "9": 51.73628070822568,
#         "10": 40.380553877503864,
#         "11": 57.41855772344896,
#         "12": 77.74101013680726,
#         "13": 30.73550312555708,
#         "14": 62.54423579388769,
#         "15": 38.800487585386264,
#         "16": 15.062075761584019,
#         "17": 79.03933793590109,
#         "18": 114.60335870872564,
#         "19": 61.65257516248233,
#         "20": 25.302769887900876,
#         "21": 15.317672731983397,
#         "22": 19.045761339026384,
#         "23": -3.7091907118020373,
#         "24": 19.874319556226006
#     },
#     "Scenario8": {
#         "Probability": 0.08527287921186018,
#         "1": 47.316085158528665,
#         "2": 37.44621405731088,
#         "3": 128.90399837176076,
#         "4": 50.64379471336946,
#         "5": 96.30965212639605,
#         "6": 60.89483394220605,
#         "7": 50.52317275893367,
#         "8": 48.80090849311462,
#         "9": 51.73628070822568,
#         "10": 11.252439104039858,
#         "11": 30.013131422145626,
#         "12": 54.33958361798484,
#         "13": 80.676349989592,
#         "14": 62.54423579388769,
#         "15": 38.800487585386264,
#         "16": 15.062075761584019,
#         "17": 79.03933793590109,
#         "18": 58.94299659421233,
#         "19": 61.65257516248233,
#         "20": 66.25964818547268,
#         "21": 15.317672731983397,
#         "22": 19.045761339026384,
#         "23": 16.461206263821154,
#         "24": 19.874319556226006
#     },
#     "Scenario9": {
#         "Probability": 0.136488354684752,
#         "1": 47.316085158528665,
#         "2": 37.44621405731088,
#         "3": 163.24303636076635,
#         "4": 19.875702035536335,
#         "5": 70.68836375127425,
#         "6": 87.14486384867749,
#         "7": 25.93390124778211,
#         "8": 22.322366635072974,
#         "9": 51.73628070822568,
#         "10": 69.50866865096788,
#         "11": 30.013131422145626,
#         "12": 30.938157099162424,
#         "13": 55.705926557574536,
#         "14": 62.54423579388769,
#         "15": 38.800487585386264,
#         "16": 15.062075761584019,
#         "17": 50.95192312182769,
#         "18": 58.94299659421233,
#         "19": 61.65257516248233,
#         "20": 25.302769887900876,
#         "21": 15.317672731983397,
#         "22": -15.05344197014503,
#         "23": 36.631603239444345,
#         "24": 19.874319556226006
#     },
#     "Scenario10": {
#         "Probability": 0.0641986951102311,
#         "1": 4.426794031862734,
#         "2": 37.44621405731088,
#         "3": 128.90399837176076,
#         "4": 19.875702035536335,
#         "5": 45.06707537615247,
#         "6": 87.14486384867749,
#         "7": 50.52317275893367,
#         "8": 48.80090849311462,
#         "9": 76.67436452309214,
#         "10": 69.50866865096788,
#         "11": 57.41855772344896,
#         "12": 30.938157099162424,
#         "13": 30.73550312555708,
#         "14": 62.54423579388769,
#         "15": 13.405340788626027,
#         "16": 43.50519539657556,
#         "17": 50.95192312182769,
#         "18": 58.94299659421233,
#         "19": 38.12994744484017,
#         "20": 25.302769887900876,
#         "21": 15.317672731983397,
#         "22": 1.9961596844406841,
#         "23": 36.631603239444345,
#         "24": 19.874319556226006
#     }
# }

# Die Probability liegt wie sichtbar in dieser .json Datei 

# Berechne mir jetzt den Gewichtet Zielfunktionswert für jeden Tag und gib ihn mir aus 
# Orientiere dich an der Struktur des bisherigen Skripts und erweitere es entsprechend 

In [45]:
def get_deterministic_filename(start_date: str, end_date: str, period: str) -> str:
    """
    Generate the deterministic filename based on the provided dates and period.
    """
    filename = f"d_weighted_{start_date}_to_{end_date}_{period}_ts.csv"
    return filename

def get_stochastic_filename(start_date: str, end_date: str, period: str, scenario: int) -> str:
    """
    Generate the stochastic filename based on the provided dates, period, and scenario number.
    """
    filename = f"s_weighted_{start_date}_to_{end_date}_{period}_{scenario}_ts.csv"
    return filename

def load_csv_data(filepath: str) -> pd.DataFrame:
    """
    Load CSV data from the given file path.
    """
    try:
        data = pd.read_csv(filepath)
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return pd.DataFrame()  # Return empty DataFrame if file not found

def load_deterministic_data(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    """
    Load deterministic data for a specific date and period.
    """
    filename = get_deterministic_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    data = load_csv_data(filepath)
    return data

def load_stochastic_data(date_str: str, period: str, scenario: int, input_path: str) -> pd.DataFrame:
    """
    Load stochastic data for a specific date, period, and scenario.
    """
    filename = get_stochastic_filename(date_str, date_str, period, scenario)
    filepath = os.path.join(input_path, filename)
    data = load_csv_data(filepath)
    return data

def load_data_for_dates(dates: List[datetime], period: str, input_path_d: str, input_path_s: str, num_scenarios: int = 10) -> Tuple[Dict[str, pd.DataFrame], Dict[str, Dict[int, pd.DataFrame]]]:
    """
    Load deterministic and stochastic data for specified dates.
    
    Returns:
        - Dictionary of deterministic data indexed by date string.
        - Nested dictionary of stochastic data indexed by date string and scenario number.
    """
    deterministic_data = {}
    stochastic_data = {}

    for date in dates:
        # Convert date to string in 'YYYYMMDD' format
        date_str = date.strftime('%Y%m%d')
        
        # Load deterministic data
        det_data = load_deterministic_data(date_str, period, input_path_d)
        deterministic_data[date_str] = det_data
        
        # Load stochastic data for each scenario
        stochastic_data[date_str] = {}
        for scenario in range(1, num_scenarios + 1):
            scen_name = f"Scenario{scenario}"
            sto_data = load_stochastic_data(date_str, period, scen_name, input_path_s)
            stochastic_data[date_str][scenario] = sto_data

    return deterministic_data, stochastic_data


def get_objective_values_filename(start_date: str, end_date: str, period: str) -> str:
    filename = f"d_scenarios_{start_date}_to_{end_date}_{period}_obj.csv"
    return filename

def load_objective_values(date_str: str, period: str, input_path: str) -> pd.DataFrame:
    filename = get_objective_values_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    data = load_csv_data(filepath)
    return data

def get_probabilities_filename(start_date: str, end_date: str, period: str) -> str:
    filename = f"reduced_heat_demand_scenarios_{start_date}_to_{end_date}_{period}.json"
    return filename

def load_probabilities(date_str: str, period: str, input_path: str) -> Dict[str, float]:
    filename = get_probabilities_filename(date_str, date_str, period)
    filepath = os.path.join(input_path, filename)
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
        probabilities = {}
        for scenario_name, scenario_data in data.items():
            probabilities[scenario_name] = scenario_data.get('Probability', 0)
        return probabilities
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return {}

def compute_weighted_objective_value(date_str: str, period: str, input_path_ws: str, input_path_hd: str) -> float:
    obj_values_df = load_objective_values(date_str, period, input_path_ws)
    if obj_values_df.empty:
        print(f"No objective values data available for date {date_str}.")
        return None
    print(obj_values_df)
    
    probabilities = load_probabilities(date_str, period, input_path_hd)
    if not probabilities:
        print(f"No probabilities data available for date {date_str}.")
        return None
    print(probabilities)

    total_weighted_objective = 0.0
    total_probability = 0.0

    for index, row in obj_values_df.iterrows():
        scenario_name = row['Scenario']
        objective_value = row['ObjectiveValue']
        probability = probabilities.get(scenario_name, 0)
        total_weighted_objective += probability * objective_value
        total_probability += probability

    if total_probability == 0:
        print(f"Total probability is zero for date {date_str}.")
        return None

    return total_weighted_objective


In [46]:
# Specify the dates you want to process
# dates_to_process = [
#     datetime(2023, 3, 3),
#     datetime(2023, 6, 26),
#     # Add more dates as needed
# ]

dates_to_process = [
    datetime(2023, 6, 26),
    # Add mo
]
# Specify the period ('day', 'week', etc.)
period = 'day'

# Load the data
deterministic_data, stochastic_data = load_data_for_dates(
    dates=dates_to_process,
    period=period,
    input_path_d=INPUT_PATH_D,
    input_path_s=INPUT_PATH_S,
    num_scenarios=10
)

In [47]:
# Choose the date to analyze
analysis_date = '20230303'  # Format: 'YYYYMMDD'


# Retrieve deterministic data for the chosen date
det_data = deterministic_data.get(analysis_date)

# Initialize a list to collect DataFrames
difference_list = []

if det_data is not None and not det_data.empty:
    # Extract the deterministic 'heat_demand' column
    det_heat_demand = det_data['heat_demand'].reset_index(drop=True)
    det_heat_demand.name = 'deterministic_heat_demand'
    
    # Initialize a DataFrame with time steps and deterministic 'heat_demand'
    comparison_df = pd.DataFrame({
        't': det_data['t'],
        'deterministic_heat_demand': det_heat_demand
    })
    
    # Loop through each scenario
    for scenario in range(1, 11):
        sto_data = stochastic_data[analysis_date].get(scenario)
        
        if sto_data is not None and not sto_data.empty:
            # Extract the 'heat_demand_scenario' column
            scenario_col_name = f'Scenario{scenario}.heat_demand_scenario'
            sto_heat_demand = sto_data[scenario_col_name].reset_index(drop=True)
            sto_heat_demand.name = f'scenario{scenario}_heat_demand_scenario'
            
            # Compute the difference
            difference = det_heat_demand - sto_heat_demand
            difference.name = f'scenario{scenario}_difference'
            
            # Add the scenario 'heat_demand_scenario' and difference to the comparison DataFrame
            comparison_df[f'scenario{scenario}_heat_demand_scenario'] = sto_heat_demand
            comparison_df[f'scenario{scenario}_difference'] = difference
        else:
            print(f"No stochastic data available for date {analysis_date}, Scenario {scenario}.")
    
    # Display the combined DataFrame
    display(comparison_df.head())
else:
    print(f"No deterministic data available for date {analysis_date}.")

No deterministic data available for date 20230303.


In [48]:
# Add a new set of columns for absolute differences
for scenario in range(1, 11):
    diff_col = f'scenario{scenario}_difference'
    abs_diff_col = f'scenario{scenario}_abs_difference'
    if diff_col in comparison_df.columns:
        # Compute absolute difference
        comparison_df[abs_diff_col] = comparison_df[diff_col].abs()
    else:
        print(f"No difference data available for Scenario {scenario}.")

# Calculate cost per hour for each scenario
for scenario in range(1, 11):
    abs_diff_col = f'scenario{scenario}_abs_difference'
    cost_col = f'scenario{scenario}_cost_per_hour'
    if abs_diff_col in comparison_df.columns:
        # Calculate cost per hour
        comparison_df[cost_col] = comparison_df[abs_diff_col] * 0.01 
    else:
        print(f"No absolute difference data available for Scenario {scenario}.")

# Initialize a dictionary to store total costs
total_costs = {}

# Sum costs over all hours for each scenario
for scenario in range(1, 11):
    cost_col = f'scenario{scenario}_cost_per_hour'
    if cost_col in comparison_df.columns:
        # Sum the costs
        total_cost = comparison_df[cost_col].sum()
        total_costs[f'Scenario {scenario}'] = total_cost
    else:
        print(f"No cost data available for Scenario {scenario}.")

total_costs

{'Scenario 1': 5.280882606575306,
 'Scenario 2': 5.674344224261326,
 'Scenario 3': 6.354211944545325,
 'Scenario 4': 4.311525142123321,
 'Scenario 5': 4.267577898609753,
 'Scenario 6': 4.026098509520683,
 'Scenario 7': 4.780717356957502,
 'Scenario 8': 5.38432227311877,
 'Scenario 9': 5.711623639642635,
 'Scenario 10': 4.095325208011705}

In [49]:
# Example of how to calculate total costs for each scenario

# Create a DataFrame to display total costs
total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Cost Dispatch'])



total_costs_df['Base Cost'] = -282.0652478293156

total_costs_df['Total Cost'] = total_costs_df['Cost Dispatch'] + total_costs_df['Base Cost']
# Display the total costs DataFrame
display(total_costs_df)



,Scenario,Cost Dispatch,Base Cost,Total Cost
0,Scenario 1,5.280883,-282.065248,-276.784365
1,Scenario 2,5.674344,-282.065248,-276.390904
2,Scenario 3,6.354212,-282.065248,-275.711036
3,Scenario 4,4.311525,-282.065248,-277.753723
4,Scenario 5,4.267578,-282.065248,-277.797670
5,Scenario 6,4.026099,-282.065248,-278.039149
6,Scenario 7,4.780717,-282.065248,-277.284530
7,Scenario 8,5.384322,-282.065248,-276.680926
8,Scenario 9,5.711624,-282.065248,-276.353624
9,Scenario 10,4.095325,-282.065248,-277.969923


In [50]:
for date in dates_to_process:
    # Convert date to string in 'YYYYMMDD' format
    analysis_date = date.strftime('%Y%m%d')
    
    print(f"Processing date: {analysis_date}")
    
    # Retrieve deterministic data for the chosen date
    det_data = deterministic_data.get(analysis_date)
    
    # Check if data exists
    if det_data is not None and not det_data.empty:
        # Initialize a DataFrame with time steps and deterministic 'heat_demand'
        det_heat_demand = det_data['heat_demand'].reset_index(drop=True)
        det_heat_demand.name = 'deterministic_heat_demand'
        comparison_df = pd.DataFrame({
            't': det_data['t'],
            'deterministic_heat_demand': det_heat_demand
        })
        
        # Loop through each scenario for the date
        for scenario in range(1, 11):
            sto_data = stochastic_data[analysis_date].get(scenario)
            if sto_data is not None and not sto_data.empty:
                # Extract the 'heat_demand_scenario' column
                scenario_col_name = f'Scenario{scenario}.heat_demand_scenario'
                sto_heat_demand = sto_data[scenario_col_name].reset_index(drop=True)
                sto_heat_demand.name = f'scenario{scenario}_heat_demand_scenario'
                
                # Compute the difference
                difference = det_heat_demand - sto_heat_demand
                difference.name = f'scenario{scenario}_difference'
                
                # Add the scenario 'heat_demand_scenario' and difference to the comparison DataFrame
                comparison_df[f'scenario{scenario}_heat_demand_scenario'] = sto_heat_demand
                comparison_df[f'scenario{scenario}_difference'] = difference
            else:
                print(f"No stochastic data available for date {analysis_date}, Scenario {scenario}.")
        
        # Perform cost calculations as before
        # Step 1: Compute absolute differences
        for scenario in range(1, 11):
            diff_col = f'scenario{scenario}_difference'
            abs_diff_col = f'scenario{scenario}_abs_difference'
            if diff_col in comparison_df.columns:
                comparison_df[abs_diff_col] = comparison_df[diff_col].abs()
            else:
                print(f"No difference data available for Scenario {scenario}.")
        
        # Step 2: Calculate cost per hour for each scenario
        for scenario in range(1, 11):
            abs_diff_col = f'scenario{scenario}_abs_difference'
            cost_col = f'scenario{scenario}_cost_per_hour'
            if abs_diff_col in comparison_df.columns:
                comparison_df[cost_col] = comparison_df[abs_diff_col] * 0.01
            else:
                print(f"No absolute difference data available for Scenario {scenario}.")
        
        # Step 3: Sum costs over all hours for each scenario
        total_costs = {}
        for scenario in range(1, 11):
            cost_col = f'scenario{scenario}_cost_per_hour'
            if cost_col in comparison_df.columns:
                total_cost = comparison_df[cost_col].sum()
                total_costs[f'Scenario {scenario}'] = total_cost
            else:
                print(f"No cost data available for Scenario {scenario}.")
        
        # Step 4: Display total costs
        total_costs_df = pd.DataFrame(list(total_costs.items()), columns=['Scenario', 'Total Cost'])
        print(f"Total Costs for date {analysis_date}:")
        display(total_costs_df)
    else:
        print(f"No deterministic data available for date {analysis_date}.")


Processing date: 20230626
Total Costs for date 20230626:


,Scenario,Total Cost
0,Scenario 1,5.280883
1,Scenario 2,5.674344
2,Scenario 3,6.354212
3,Scenario 4,4.311525
4,Scenario 5,4.267578
5,Scenario 6,4.026099
6,Scenario 7,4.780717
7,Scenario 8,5.384322
8,Scenario 9,5.711624
9,Scenario 10,4.095325


In [51]:
# Initialize a dictionary to store weighted objective values
weighted_objective_values = {}

for date in dates_to_process:
    date_str = date.strftime('%Y%m%d')
    print(f"\nProcessing date: {date_str}")

    weighted_obj_value = compute_weighted_objective_value(
        date_str=date_str,
        period=period,
        input_path_ws=INPUT_PATH_WS,
        input_path_hd=INPUT_PATH_HD
    )

    if weighted_obj_value is not None:
        weighted_objective_values[date_str] = weighted_obj_value
        print(f"Weighted objective value for date {date_str}: {weighted_obj_value:.2f}")
    else:
        print(f"Could not compute weighted objective value for date {date_str}.")

# Display the results
weighted_obj_values_df = pd.DataFrame(
    list(weighted_objective_values.items()),
    columns=['Date', 'Weighted Objective Value']
)
weighted_obj_values_df['Date'] = pd.to_datetime(weighted_obj_values_df['Date'], format='%Y%m%d').dt.date

print("\nWeighted Objective Function Values:")
display(weighted_obj_values_df)


Processing date: 20230626
     Scenario  ObjectiveValue
0   Scenario1     -115.799912
1   Scenario2     -125.273875
2   Scenario3      -98.026980
3   Scenario4     -106.034452
4   Scenario5     -110.521314
5   Scenario6     -109.615761
6   Scenario7     -117.038037
7   Scenario8     -136.101482
8   Scenario9     -112.079350
9  Scenario10     -112.687763
{'Scenario1': 0.2418623085247977, 'Scenario2': 0.07995197538112958, 'Scenario3': 0.059835692794664365, 'Scenario4': 0.061774261157197925, 'Scenario5': 0.07925989359232213, 'Scenario6': 0.07113913002121826, 'Scenario7': 0.12021680952182671, 'Scenario8': 0.08527287921186018, 'Scenario9': 0.136488354684752, 'Scenario10': 0.0641986951102311}
Weighted objective value for date 20230626: -115.20

Weighted Objective Function Values:


,Date,Weighted Objective Value
0,2023-06-26,-115.204756
